In [ ]:
'''
L'objectif de ce projet est d'observer s'il existe des 'déserts sportifs', 
lieux en France où les infrastrcutures sportives manquent. 
La question sera alors d'essayer d'expliquer ces déserts sportifs, que ce soit par des raisons économiques ou politiques. 
Enfin, il s'agira de comparer la carte des déserts sportifs avec d'autres cartes connues, à l'instar des déserts médicaux.
'''

In [8]:
''' 
Il convient d'abord d'importer tous les modules python nécessaires au travail
'''

import numpy as np
import numpy.linalg as al
import matplotlib.pyplot as plt 
import pandas as pd
import geopandas as gpd
from cartiflette import carti_download
import requests
import zipfile
import io

In [63]:
'''
On importe ensuite nos jeux de données
Le jeu de données principal
'''
url = "https://data.sports.gouv.fr/api/explore/v2.1/catalog/datasets/equipements-sportifs/exports/csv?lang=fr&timezone=Europe%2FBerlin&use_labels=true&delimiter=%3B"
equipement = pd.read_csv(url, sep=";", low_memory=False)

'''
Un jeu de données csv sur des informations économiques et démographiques au niveau des communes
'''

urlpop = "https://www.insee.fr/fr/statistiques/fichier/2521169/base_cc_comparateur_csv.zip"
response = requests.get(urlpop)

with zipfile.ZipFile(io.BytesIO(response.content)) as z:
    with z.open("base_cc_comparateur.csv") as csvfile:
        df_communes = pd.read_csv(csvfile, sep=";", low_memory=False)

'''
Deux jeux de données politiques au niveau des communes: résultats des législatives 2024 des 1er et 2nd tours
'''

url1="https://www.data.gouv.fr/api/1/datasets/r/bd32fcd3-53df-47ac-bf1d-8d8003fe23a1"
url2="https://www.data.gouv.fr/api/1/datasets/r/5a8088fd-8168-402a-9f40-c48daab88cd1"

legislatives1=pd.read_csv(url1, sep=";", low_memory=False)
legislatives2=pd.read_csv(url2, sep=";", low_memory=False)


In [64]:
'''
Il s'agit maintenant de laver nos jeux de données afin de les réunir ensuite à l'aide des codes INSEE.
On conserve seulement les variables qui peuvent nous intéresser.
'''

cols = ["Nom de l'installation sportive", "Code Postal", "Commune nom", "Commune INSEE", "Département Code", "Département Nom", "Densite Catégorie", "Nom de l'équipement sportif", "Type d'équipement sportif", "Longitude", "Latitude"]
equipement = equipement[cols]

cols=["CODGEO", "P22_POP", "NAISD24", "DECESD24", "P22_MEN","MED21", "TP6021", "P22_CHOM1564"]
df_communes = df_communes[cols]


cols=["Code commune", "Libellé commune"]
cols= cols + [(f"% Voix/exprimés {i}") for i in range(1,9)]+[ (f"Nuance candidat {i}") for i in range(1, 9)]
legislatives1= legislatives1[cols]

cols=["Code commune", "Libellé commune"]
cols= cols + [(f"% Voix/exprimés {i}") for i in range(1,4)]+[ (f"Nuance candidat {i}") for i in range(1, 4)]
legislatives2= legislatives2[cols]

In [80]:
'''
On renomme maintenant les colonnes afin de pouvoir concatener les dataframes. 
'''

df_communes = df_communes.rename(columns={"CODGEO": "Commune INSEE"})
legislatives1 = legislatives1.rename(columns={"Code commune": "Commune INSEE"})
legislatives2 = legislatives2.rename(columns={"Code commune": "Commune INSEE"})

df_final = (
    equipement
    .merge(df_communes, on="Commune INSEE", how="left")
    .merge(legislatives2, on="Commune INSEE", how="left")
)

print(df_final.head())

       Nom de l'installation sportive  Code Postal     Commune nom  \
0                   TERRAIN DE TENNIS        23110          Sannat   
1                     STADE MUNICIPAL        23250         Sardent   
2                CHEMINS DE RANDONNEE        23600         Soumans   
3                STADE EUGENE ROMAINE        23600         Soumans   
4  LOCAUX ASSOCIATIFS DU PUY CHARRAUD        23300  La Souterraine   

  Commune INSEE Département Code Département Nom  \
0         23167               23          Creuse   
1         23168               23          Creuse   
2         23174               23          Creuse   
3         23174               23          Creuse   
4         23176               23          Creuse   

               Densite Catégorie Nom de l'équipement sportif  \
0  Rural à habitat très dispersé         TERRAIN MULTISPORTS   
1  Rural à habitat très dispersé         TERRAIN DE FOOTBALL   
2  Rural à habitat très dispersé  CHEMIN DU LAVOIR A CHAZEIX   
3  Rural à

In [79]:
'''
On ajoute dans ce dataframe uniquement le parti politique en tête lors des élections de 2024, c'est-à-dire la nuance qui a eu
le plus de % Voix/exprimés
'''

cols_voix = [f"% Voix/exprimés {i}" for i in range(1,4)]
equipement["Voix_max"] = equipement[cols_voix].max()
equipement["Liste_gagnante_num"] = equipement[cols_voix].idxmax(axis=1).str.extract("(\d+)").astype(float)
print(equipement)

<>:8: SyntaxWarning: invalid escape sequence '\d'
<>:8: SyntaxWarning: invalid escape sequence '\d'
/tmp/ipykernel_1110/952324450.py:8: SyntaxWarning: invalid escape sequence '\d'
  equipement["Liste_gagnante_num"] = equipement[cols_voix].idxmax(axis=1).str.extract("(\d+)").astype(float)
/tmp/ipykernel_1110/952324450.py:8: SyntaxWarning: invalid escape sequence '\d'
  equipement["Liste_gagnante_num"] = equipement[cols_voix].idxmax(axis=1).str.extract("(\d+)").astype(float)


KeyError: "None of [Index(['% Voix/exprimés 1', '% Voix/exprimés 2', '% Voix/exprimés 3'], dtype='object')] are in the [columns]"